This script impute Supplemental Security Income (SSI) recipients and dollar benefit amount to match the aggregates with Social Security Administration (SSA) statistics for SSI. In this current version, we used 2014 CPS data and SSA [2014 annual reports on SSI](https://www.ssa.gov/policy/docs/statcomps/ssi_asr/). Please refer to the documentation in the same folder for more details on methodology and assumptions. The output this script is a personal level dataset that contains CPS personal level ID (PERIDNUM), individual participation indicator (SSI_participation, 0 - not a recipient, 1 - current recipient on file, 2 - imputed recipient), and benefit amount.

Input: 2014 CPS (cpsmar2014t.csv), 2014 number of recipients by age and state in Dec 2014 (SSI_by_age_state.csv)

Output: SSI_Imputation.csv

Additional Source links: https://www.ssa.gov/policy/docs/statcomps/ssi_sc/2014/table02.html

In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import random
import statsmodels.formula.api as sm

Trim the dataset and only keep columns relevant to SSI augmentation

In [ ]:
CPS_dataset = pd.read_csv('cpsmar2014t.csv')

In [ ]:
columns_to_keep = ['ssi_val', 'ssi_yn','ssikidyn', 'resnssi1', 'resnssi2', 'marsupwt', 'a_age', 'gestfips',
                   'pedisdrs', 'pedisear', 'pediseye', 'pedisout', 'pedisphy', 'pedisrem',
                   'dis_hp', 'rsnnotw', 'vet_typ1', 'pemlr', 'mcare', 'a_ftpt', 'filestat',
                   'wsal_val', 'semp_val', 'frse_val', 'ss_val', 'rtm_val', 'oi_off', 'oi_val',
                   'uc_yn', 'uc_val', 'int_yn', 'int_val', 'a_spouse', 'paw_yn', 'vet_yn', 
                   'ffpos', 'fh_seq', 'finc_ssi', 'ftot_r', 'ftotval', 'ptot_r', 'ptotval',
                   'peridnum']

CPS_dataset = CPS_dataset[columns_to_keep]
CPS_dataset.to_csv('CPSASEC_SSI.csv', columns=columns_to_keep, index=False)

In [45]:
CPS_dataset = pd.read_csv('CPSASEC_SSI.csv')

# Converting age and SSI benefit to numeric values

In [46]:
# for now assuming ages are evenly distributed in that range
CPS_dataset.a_age = np.where(CPS_dataset.a_age == "80-84 years of age",
                             random.randrange(80, 84),
                             CPS_dataset.a_age)
CPS_dataset.a_age = np.where(CPS_dataset.a_age == "85+ years of age",
                             random.randrange(85, 95),
                             CPS_dataset.a_age)
CPS_dataset.a_age = pd.to_numeric(CPS_dataset.a_age)

In [47]:
# all existing ssi_val are greater than 0 (min==1)
CPS_dataset.ssi_val = np.where(CPS_dataset.ssi_val == "None or not in universe",'0', CPS_dataset.ssi_val)
CPS_dataset.ssi_val = pd.to_numeric(CPS_dataset.ssi_val)

# Create disability proxy indicator

In [48]:
a1 = (CPS_dataset.pedisdrs == 'Yes')
a2 = (CPS_dataset.pedisear == 'Yes')
a3 = (CPS_dataset.pediseye == 'Yes')
a4 = (CPS_dataset.pedisout == 'Yes')
a5 = (CPS_dataset.pedisphy == 'Yes')
a6 = (CPS_dataset.pedisrem == 'Yes')
disability = (a1|a2|a3|a4|a5|a6)

In [49]:
d1 = (CPS_dataset.dis_hp == 'Yes')
d2 = (CPS_dataset.pemlr == 'Not in labor force - disabled')
d3 = (CPS_dataset.rsnnotw == 'Ill or disabled')
d4 = (CPS_dataset.vet_typ1 == 'Yes')
d5 = (CPS_dataset.a_age < 65) & (CPS_dataset.mcare == 'Yes')
work_disability = (d1|d2|d3|d4|d5)

# Create SSI countable income proxy

In [50]:
def countable(earned, unearned, deemed=None):
    if deemed is not None:
        SSI_countable = earned + unearned + deemed
    else:
        SSI_countable = earned + unearned
        
    SSI_countable = np.where(SSI_countable > 0, SSI_countable, 0)
    
    # disgards
    SSI_countable = np.where(SSI_countable>20*12, SSI_countable - 20*12, 0) # exclude $20 from most income items
    SSI_countable = np.where(earned > 65*12, SSI_countable - 65*12, 0) # exclude he first $65 of earnings 
    SSI_countable = np.where(earned > 65*12, SSI_countable - 0.5*(earned - 65*12), 0) # one–half of earnings over $65 received in a month 
    
    return SSI_countable

In [51]:
wage = pd.to_numeric(np.where(CPS_dataset.wsal_val!= 'None or not in universe', CPS_dataset.wsal_val, 0))
self_employed1 = pd.to_numeric(np.where(CPS_dataset.semp_val!= 'None or not in universe', CPS_dataset.semp_val, 0))
self_employed2 = pd.to_numeric(np.where(CPS_dataset.frse_val!= 'None or not in universe', CPS_dataset.frse_val, 0))
earned = wage + self_employed1 + self_employed2

In [52]:
ss = pd.to_numeric(np.where(CPS_dataset.ss_val!='None or not in universe', CPS_dataset.ss_val, 0))
pension = pd.to_numeric(np.where(CPS_dataset.rtm_val!='None or not in universe', CPS_dataset.rtm_val, 0))
#too under-reported?
disability = pd.to_numeric(np.where(CPS_dataset.oi_off=='State disability payments', CPS_dataset.oi_val, 0))
unemploy = pd.to_numeric(np.where(CPS_dataset.uc_yn=='Yes', CPS_dataset.uc_val, 0))
interest = pd.to_numeric(np.where(CPS_dataset.int_yn=='Yes', CPS_dataset.int_val, 0))
unearned = ss + pension + disability + unemploy + interest

In [53]:
no_public_assistance = (CPS_dataset.paw_yn != 'Yes')
not_married_HH = (CPS_dataset.filestat=='Single')|(CPS_dataset.filestat=='Nonfiler')
student_under22 = (CPS_dataset.a_ftpt != 'Not in universe or children and')&(CPS_dataset.a_age < 22)
under_18 = (CPS_dataset.a_age < 18)
ineligible_children = np.where(no_public_assistance&(student_under22|under_18), 1, 0)

In [54]:
SSI_countable = countable(earned, unearned)

In [55]:
combined = np.where((disability==1)|(work_disability==1), 1, 0)
aged = np.where(CPS_dataset.a_age>=65, 1, 0)

In [56]:
subfields = {'old_disabled': combined&aged,
             'young_disabled': combined&(1-aged),
             'old_not_disabled': (1-combined)&aged,
             'young_not_disabled': (1-combined)&(1-aged)}

In [57]:
low_income = np.zeros(len(CPS_dataset))
for key in subfields:
    benchmark = np.percentile(SSI_countable[CPS_dataset.index[(CPS_dataset.ssi_yn=='Yes')&subfields[key]]], 95)
    low_income = np.where((SSI_countable < benchmark) & subfields[key], 1, 0)
    print "group: %s, benchmark: %d" % (key, benchmark)

group: young_disabled, benchmark: 429
group: old_not_disabled, benchmark: 0
group: old_disabled, benchmark: 0
group: young_not_disabled, benchmark: 15607


In [58]:
CPS_dataset['low_income'] = low_income
CPS_dataset['ineligible_children'] = ineligible_children
CPS_dataset['earned'] = earned
CPS_dataset['unearned'] = unearned

# Defining SSI target records

In [59]:
SSI_target_pool = np.zeros(len(CPS_dataset))
SSI_target_pool = np.where(low_income&disability, 1, 0)
SSI_target_pool = np.where((work_disability&low_income), 1, SSI_target_pool)
SSI_target_pool = np.where(low_income&(CPS_dataset.a_age<18), 1, SSI_target_pool)
SSI_target_pool = np.where(low_income&(CPS_dataset.a_age>=65), 1, SSI_target_pool)

In [60]:
CPS_dataset['current_recipient'] = np.where((CPS_dataset.ssi_yn =='Yes')|(CPS_dataset.ssikidyn=='Received SSI'), 1, 0)
SSI_target_pool = np.where(CPS_dataset.current_recipient==1, 1, SSI_target_pool)

In [61]:
CPS_dataset['SSI_target_pool'] = SSI_target_pool

# import administrative totals

In [62]:
admin = pd.read_csv("SSI_by_age_state.csv", 
                    dtype={'Under 18': np.float, '18-64':np.float, '65 or older': np.float,
                           'Under 18 mean': np.float, '18-64 mean':np.float, '65 or older mean': np.float})
admin.index = admin.Fips

In [63]:
# adjust targets and export adjusted targets to a CSV file
state_benefit = {}
state_recipients = {}
for state in admin.Fips:
    this_state = (CPS_dataset.gestfips==state)
    CPS_total = (CPS_dataset.ssi_val * CPS_dataset.marsupwt)[this_state].sum()/1000000
    federal_total = admin['State Total'][state]/1000
    federal_total_adj = admin['State Total'][state] * 12.096/1000
    
    state_state = admin["State-administered Supplement 2010"][state]/1000000
    state_state_adj = admin["State-administered Supplement 2010"][state] * 721/674/1000000
    admin_total = federal_total_adj + state_state_adj
    
    temp = [admin['State'][state], CPS_total, federal_total, federal_total_adj, 
            state_state, state_state_adj, admin_total]
    state_benefit[state] = temp
    
pre_augment_benefit = DataFrame(state_benefit).transpose()
pre_augment_benefit.columns = ['State', 'CPS total', 'Federal Total', 'Federal Total after Adjustment',
                               'State Total', 'State Total After adjustment', 'Admin Total']

In [64]:
pre_augment_benefit.to_csv('pre-blow-up.csv')

In [65]:
# caculate difference of SSA stats and CPS aggregates on recipients number
# by state and age
diff = {'Fips': [], 'Under 18':[], '18-64':[], '65 or older':[],
        'Under 18 mean':[], '18-64 mean':[], '65 or older mean':[],
       'Under 18 cps':[], '18-64 cps':[], '65 or older cps':[],
       'Under 18 ssa':[], '18-64 ssa':[], '65 or older ssa':[]}
diff['Fips'] = admin.Fips

current = (CPS_dataset.current_recipient==1)
category = {'Under 18': (CPS_dataset.a_age<18),
            '18-64': (CPS_dataset.a_age>=18)&(CPS_dataset.a_age<65),
            '65 or older': (CPS_dataset.a_age>=65)}

augment_ratio = {'Under 18': 1.401/1.299,
                 '18-64': 5.523/4.626,
                 '65 or older': 2.344/2.122}

for group in ['Under 18', '18-64', '65 or older']:
    for FIPS in admin.Fips:
        this_state = (CPS_dataset.gestfips==FIPS)
        age_range = category[group]
        current_tots = CPS_dataset.marsupwt[current&this_state&age_range].sum()
        
        valid_num = CPS_dataset.marsupwt[current&this_state&age_range].sum() + 0.0000001
        current_mean = ((CPS_dataset.ssi_val*CPS_dataset.marsupwt)[current&this_state&age_range]).sum()/valid_num
        
        this_admin = augment_ratio[group] * float(admin[group][admin.Fips==FIPS])
        diff[str(group) + ' cps'].append(current_tots)
        diff[str(group) + ' ssa'].append(float(admin[group][admin.Fips==FIPS]))
        diff[group].append(this_admin - current_tots)
        diff[str(group + ' mean')].append(float(current_mean)/12)

In [66]:
d = DataFrame(diff)
d.to_csv('recipients.csv', index=False)

In [67]:
CPS_dataset['ssi_indicator'] = np.where((CPS_dataset.ssi_yn=='Yes')|(CPS_dataset.ssikidyn=='Received SSI'), 1, 0)
CPS_dataset['countable_income'] = SSI_countable

SSI_disability = ((CPS_dataset.resnssi1!='NIU')&(CPS_dataset.resnssi1!='Other (adult or child)'))|((CPS_dataset.resnssi2!='NIU')&(CPS_dataset.resnssi2!='Other (adult or child)'))
CPS_dataset['combined_disability'] = np.where(disability|work_disability|SSI_disability, 1, 0)

model = sm.ols(formula='ssi_indicator ~ countable_income + combined_disability + a_age', 
               data=CPS_dataset)
results = model.fit()

In [68]:
probs = results.fittedvalues

In [69]:
CPS_dataset['impute'] = np.zeros(len(CPS_dataset))
CPS_dataset['ssi_impute'] = np.zeros(len(CPS_dataset))

non_current = (CPS_dataset.current_recipient == 0)
current = (CPS_dataset.current_recipient == 1)
random.seed()

for group in ['Under 18','18-64', '65 or older']:
    for FIPS in admin.Fips:
        
        print ('we need to impute', d[group][FIPS], 'for state', FIPS)
        
        if d[str(group)][FIPS] < 0:
            continue
        else:
            this_state = (CPS_dataset.gestfips==FIPS)
            age_range = category[group]
            
            not_imputed = (CPS_dataset.impute==0)
            
            pool_index = CPS_dataset[this_state&age_range&not_imputed&non_current].index
            pool = DataFrame({'weight': CPS_dataset.marsupwt[pool_index], 'prob': probs[pool_index]},
                            index=pool_index)
            
            pool = pool.sort(columns='prob', ascending=False)
            pool['cumsum_weight'] = pool['weight'].cumsum()
            pool['distance'] = abs(pool.cumsum_weight-d[str(group)][FIPS])
            min_index = pool.sort(columns='distance')[:1].index
            min_weight = int(pool.loc[min_index].cumsum_weight)
            pool['impute'] = np.where(pool.cumsum_weight<=min_weight+10 , 1, 0)
            CPS_dataset.impute[pool.index[pool['impute']==1]] = 1
            CPS_dataset.ssi_impute[pool.index[pool['impute']==1]] = admin[str(group)+' mean'][FIPS] * 12
            
            #if group == 'Under 18':
            #    CPS_dataset.ssi_impute[(current)&age_range&this_state] = admin[str(group)+' mean'][FIPS] * 12
        
        print ('Method1: regression gives', 
                CPS_dataset.marsupwt[(CPS_dataset.impute==1)&this_state&age_range].sum())   

/Users/Amy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:25: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/Amy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:28: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/Amy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('we need to impute', 24307.570069284066, 'for state', 1)
('Method1: regression gives', 25182.159999999996)
('we need to impute', 1356.7806004618938, 'for state', 2)
('Method1: regression gives', 1256.38)
('we need to impute', 18257.993256351037, 'for state', 4)
('Method1: regression gives', 17422.33)
('we need to impute', 12636.928752886837, 'for state', 5)
('Method1: regression gives', 11745.17)
('we need to impute', 100907.57448036951, 'for state', 6)
('Method1: regression gives', 100702.62)
('we need to impute', 7661.5727020785216, 'for state', 8)
('Method1: regression gives', 8380.79)
('we need to impute', 3768.9590993071597, 'for state', 9)
('Method1: regression gives', 3382.2999999999997)
('we need to impute', 3731.977505773672, 'for state', 10)
('Method1: regression gives', 3687.86)
('we need to impute', 3120.1509930715933, 'for state', 11)
('Method1: regression gives', 3279.9)
('we need to impute', 84804.933371824474, 'for state', 12)
('Method1: regression gives', 84504.799999

/Users/Amy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [71]:
results = {}

imputed = (CPS_dataset.impute == 1)
has_val = (CPS_dataset.ssi_val != 0)
no_val = (CPS_dataset.ssi_val == 0)

for FIPS in admin.Fips:
    this_state = (CPS_dataset.gestfips==FIPS)
    
    # sum up current recipients total and imputed totals
    current_total = (CPS_dataset.ssi_val*CPS_dataset.marsupwt)[this_state].sum()
    imputed_total = (CPS_dataset.ssi_impute*CPS_dataset.marsupwt)[this_state&imputed].sum()
    on_file = (current_total + imputed_total)/1000000
    
    # w
    federal_total = admin['State Total'][FIPS] * 12.096/1000
    state_state = admin["State-administered Supplement 2010"][FIPS] * 721/674/1000000
    admin_total = federal_total + state_state
    
    adjust_ratio = admin_total / on_file
    this_state_num = [admin['State'][FIPS], on_file, admin_total, adjust_ratio]
    results[FIPS] = this_state_num
    

    CPS_dataset.ssi_impute = np.where(has_val&this_state, CPS_dataset.ssi_val * adjust_ratio, CPS_dataset.ssi_impute)
    CPS_dataset.ssi_impute = np.where(no_val&this_state, CPS_dataset.ssi_impute * adjust_ratio, CPS_dataset.ssi_impute)

CPS_dataset["ssi_participation"] = np.zeros(len(CPS_dataset))
ssi_participation = np.where(CPS_dataset.impute==1, 2, 0)
ssi_participation = np.where(has_val, 1, CPS_dataset.ssi_participation)


r = DataFrame(results).transpose()
r.columns=['State', 'Imputed', 'Admin', 'adjust ratio']
r.to_csv('amount.csv', index=False)

In [72]:
CPS_dataset.to_csv('SSI_Imputation.csv', 
                   columns=['peridnum', 'ssi_participation', 'ssi_impute'],
                   index=False)